## Dice coeffient babyAFQ to pyAFQ

In [ ]:
def density_map(tractogram):
    """
    calculate a density map image for a given tractogram
    """
    import numpy as np
    from dipy.io.utils import create_nifti_header, get_reference_info
    import dipy.tracking.utils as dtu
    import nibabel as nib

    affine, vol_dims, voxel_sizes, voxel_order = get_reference_info(tractogram)
    tractogram_density = dtu.density_map(tractogram.streamlines, np.eye(4), vol_dims)
    tractogram_density = np.uint8(tractogram_density)
    nifti_header = create_nifti_header(affine, vol_dims, voxel_sizes)
    density_map_img = nib.Nifti1Image(tractogram_density, affine, nifti_header)

    return density_map_img

In [ ]:
def baseline_dice(test_tractogram_filename, retest_tractogram_filename):
    """
    compute the weighted dice coefficent between two tractogram files
    """
    from dipy.io.stateful_tractogram import StatefulTractogram
    from dipy.io.streamline import load_tractogram
    import nibabel as nib
    from AFQ.utils.volume import dice_coeff
    
    reference = nib.load('/Users/bloomdt/AFQ_data/kalanit/derivatives/mrtrix/sub-01/ses-01/anat/b0.nii.gz')
    test_tractogram = load_tractogram(test_tractogram_filename, reference)
    test_sft = StatefulTractogram.from_sft(test_tractogram.streamlines, test_tractogram)                        
    test_sft.to_vox()
    test_sft_map = density_map(test_sft)

    retest_tractogram = load_tractogram(retest_tractogram_filename, reference)
    retest_sft = StatefulTractogram.from_sft(retest_tractogram.streamlines, retest_tractogram)                        
    retest_sft.to_vox()
    retest_sft_map = density_map(retest_sft)

    return dice_coeff(test_sft_map, retest_sft_map)

In [ ]:
def dice(test_tractogram_filename, retest_tractogram_filename):
    """
    compute the weighted dice coefficent between two tractogram files
    """
    from dipy.io.stateful_tractogram import StatefulTractogram
    from dipy.io.streamline import load_tractogram
    import nibabel as nib
    from AFQ.utils.volume import dice_coeff
    
    reference = nib.load('/Users/bloomdt/AFQ_data/kalanit/derivatives/mrtrix/sub-01/ses-01/anat/b0.nii.gz')
    test_tractogram = load_tractogram(test_tractogram_filename, reference)
    test_sft = StatefulTractogram.from_sft(test_tractogram.streamlines, test_tractogram)                        
    test_sft.to_vox()
    test_sft_map = density_map(test_sft)

    retest_tractogram = load_tractogram(retest_tractogram_filename, 'same')
    retest_sft = StatefulTractogram.from_sft(retest_tractogram.streamlines, retest_tractogram)                        
    retest_sft.to_vox()
    retest_sft_map = density_map(retest_sft)

    return dice_coeff(test_sft_map, retest_sft_map)

In [ ]:
def baseline_config():
    """
    compare babyAFQ to manual
    """
    test_dir = '/Users/bloomdt/Desktop/UW NRDG/WIP pyBabyAFQ 524/Kalanit/manual_baby_afq'
    retest_dir = '/Users/bloomdt/Desktop/UW NRDG/WIP pyBabyAFQ 524/Kalanit/baby_afq'
    
    correspondances = [
        # 'Arcuate Fasciculus'
        ('ARC_L', f'lh_AF_mrtrix.tck', f'lh_AF.tck'),
        ('ARC_R', f'rh_AF_mrtrix.tck', f'rh_AF.tck'),

        # 'Thalamic Radiation'
        ('ATR_L', f'lh_TR_mrtrix.tck', f'lh_TR.tck'),
        ('ATR_R', f'rh_TR_mrtrix.tck', f'rh_TR.tck'),

        # 'Cingulum Cingulate'
        ('CGC_L', f'lh_CC_mrtrix.tck', f'lh_CC.tck'),
        ('CGC_R', f'rh_CC_mrtrix.tck', f'rh_CC.tck'),

        # 'Corticospinal'
        ('CST_L', f'lh_CS_mrtrix.tck', f'lh_CS.tck'),
        ('CST_R', f'rh_CS_mrtrix.tck', f'rh_CS.tck'),

        # 'Forceps Major'
        ('FP', f'lh_FMa_mrtrix.tck', f'lh_FMa.tck'),

        # 'Forceps Minor'
        ('FA', f'rh_FMi_mrtrix.tck', f'rh_FMi.tck'),

        # 'Inferior Fronto-occipital'
        ('IFO_L', f'lh_IFOF_mrtrix.tck', f'lh_IFOF.tck'),
        ('IFO_R', f'rh_IFOF_mrtrix.tck', f'rh_IFOF.tck'),

        # 'Inferior Longitudinal Fasciculus'
        ('ILF_L', f'lh_ILF_mrtrix.tck', f'lh_ILF.tck'),
        ('ILF_R', f'rh_ILF_mrtrix.tck', f'rh_ILF.tck'),

        # 'Superior Longitudinal Fasciculus'
        ('SLF_L', f'lh_SLF_mrtrix.tck', f'lh_SLF.tck'),
        ('SLF_R', f'rh_SLF_mrtrix.tck', f'rh_SLF.tck'),

        # 'Uncinate Fasciculus'
        ('UNC_L', f'lh_UCI_mrtrix.tck', f'lh_UCI.tck'),
        ('UNC_R', f'rh_UCI_mrtrix.tck', f'rh_UCI.tck'),
    ]
    
    return test_dir, retest_dir, correspondances

In [ ]:
def config(expirement_name, gold=True):
    """
    get configuration variables for expirement
    
    returns tuple test directory, retest dirctory, and correspondance tuple; where 
        test directory is location of gold standard or baby afq tractogram files,
        retest directory is location of expirement tractogram files, and
        correspondance is itself a tuple of bundle name, test tractogram filename, and retest tractogram file name
    """
    retest_dir = f'/Users/bloomdt/Desktop/UW NRDG/WIP pyBabyAFQ 524/Kalanit/{expirement_name}/sub-01/ses-01/clean_bundles'

    if (gold):
        suffix = '_mrtrix'
        test_dir = '/Users/bloomdt/Desktop/UW NRDG/WIP pyBabyAFQ 524/Kalanit/manual_baby_afq'
    else:
        suffix = ''
        test_dir = '/Users/bloomdt/Desktop/UW NRDG/WIP pyBabyAFQ 524/Kalanit/baby_afq'

    correspondances = [
        # 'Arcuate Fasciculus'
        ('ARC_L', f'lh_AF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ARC_L_tractography.trk'),
        ('ARC_R', f'rh_AF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ARC_R_tractography.trk'),

        # 'Thalamic Radiation'
        ('ATR_L', f'lh_TR{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ATR_L_tractography.trk'),
        ('ATR_R', f'rh_TR{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ATR_R_tractography.trk'),

        # 'Cingulum Cingulate'
        ('CGC_L', f'lh_CC{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-CGC_L_tractography.trk'),
        ('CGC_R', f'rh_CC{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-CGC_R_tractography.trk'),

        # 'Cingulum Hippocampus' -- not included in pyAFQ
    #     (f'lh_CH{suffix}.tck', ),
    #     (f'rh_CH{suffix}.tck', ),

        # 'Corticospinal'
        ('CST_L', f'lh_CS{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-CST_L_tractography.trk'),
        ('CST_R', f'rh_CS{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-CST_R_tractography.trk'),

        # 'Forceps Major'
        ('FP', f'lh_FMa{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-FP_tractography.trk'),

        # 'Forceps Minor'
        ('FA', f'rh_FMi{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-FA_tractography.trk'),

        # 'Inferior Fronto-occipital'
        ('IFO_L', f'lh_IFOF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-IFO_L_tractography.trk'),
        ('IFO_R', f'rh_IFOF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-IFO_R_tractography.trk'),

        # 'Inferior Longitudinal Fasciculus'
        ('ILF_L', f'lh_ILF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ILF_L_tractography.trk'),
        ('ILF_R', f'rh_ILF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-ILF_R_tractography.trk'),

        # 'Posterior Arcuate Fasciculus'
    #     (f'lh_pAF_sum{suffix}.tck', ),
    #     (f'rh_pAF_sum{suffix}.tck', ),
    #     (f'lh_pAF_VOT{suffix}.tck', ),
    #     (f'rh_pAF_VOT{suffix}.tck', ),
    #     (f'lh_pAF{suffix}.tck', ),
    #     (f'rh_pAF{suffix}.tck', ),

        # 'Superior Longitudinal Fasciculus'
        ('SLF_L', f'lh_SLF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-SLF_L_tractography.trk'),
        ('SLF_R', f'rh_SLF{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-SLF_R_tractography.trk'),

        # 'Uncinate Fasciculus'
        ('UNC_L', f'lh_UCI{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-UNC_L_tractography.trk'),
        ('UNC_R', f'rh_UCI{suffix}.tck','sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-UNC_R_tractography.trk'),

        # 'Vertical Occipital Fasciculus'
    #     (f'lh_VOF{suffix}.tck', ),
    #     (f'rh_VOF{suffix}.tck', ),

        # 'Middle Longitudinal Fasciculus'
    #     (,'sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-MdLF_L_tractography.trk'),
    #     (,'sub-01_dwi_space-RASMM_model-DTI_desc-det-AFQ-MdLF_R_tractography.trk'),
    ]
    
    return test_dir, retest_dir, correspondances

In [ ]:
def baseline_bundle_dice():
    from os.path import join
    
    results = []
    
    test_dir, retest_dir, correspondances =  baseline_config()
    
    for correspondance in correspondances:
        test_filename = join(test_dir, correspondance[1])
        retest_filename = join(retest_dir, correspondance[2])
        
        results.append((correspondance[0], baseline_dice(test_filename, retest_filename)))
        
    return results

In [ ]:
def bundle_dice(expirement_name, gold=True):
    """
    calculate the dice coefficent for given expirement for all bundles
    
    by default calculates with respect to the gold standard.
    
    returns tuple with bundle name and dice coefficient
    """
    from os.path import join
    
    results = []
    
    test_dir, retest_dir, correspondances = config(expirement_name, gold)
    
    for correspondance in correspondances:
        test_filename = join(test_dir, correspondance[1])
        retest_filename = join(retest_dir, correspondance[2])
        
        results.append((correspondance[0], dice(test_filename, retest_filename)))
        
    return results

In [ ]:
def compare_to_gold(expirement_name):
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    df_baby = pd.DataFrame(baseline_bundle_dice(), columns = ['bundles', 'dice'])
    df_baby.insert(0, 'name', 'babyAFQ')

    df_py = pd.DataFrame(bundle_dice(expirement_name), columns = ['bundles', 'dice'])
    df_py.insert(0, 'name', 'pyAFQ')
    
    df_combined = pd.DataFrame()
    df_combined['bundles'] = df_baby['bundles']
    df_combined['babyAFQ dice'] = df_baby['dice']
    df_combined['pyAFQ dice'] = df_py['dice']
    display(df_combined)
    
    df_concat = pd.concat([df_baby, df_py], ignore_index=True)
    
    plt.figure(figsize=(16, 6))
    sns.barplot(x='bundles', y='dice', hue='name', data=df_concat)
    plt.title('comparison to gold standard manual segmentation pyAFQ='+expirement_name)
    
    return df_combined

In [ ]:
df = compare_to_gold('afq-ct-t2-sd2-no_rio_dilation-affine_reg')

In [ ]:
df = compare_to_gold('afq-ct-t2-sd3-no_rio_dilation-affine_reg')

In [ ]:
df.to_clipboard()